In [22]:
import os
import pandas as pd
from pandas import json_normalize
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
from tabulate import tabulate
from dotenv import load_dotenv

In [23]:
load_dotenv()
project_id = os.getenv('GOOGLE_PROJECT_ID')
key_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_path
client = bigquery.Client(project=project_id)

Read the first 10 events in "ga4_obfuscated_sample_ecommerce"

In [24]:
query = """
    SELECT *
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    LIMIT 10
"""

In [25]:
query_job = client.query(query)
results = query_job.result()
rows = [dict(row) for row in results]
df = pd.DataFrame(rows)
if 'event_params' in df.columns:
    event_params = json_normalize(df['event_params'])
    df = df.drop('event_params', axis=1).join(event_params)
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+--------------+-------------------+---------------------+----------------------------+----------------------+----------------------------+---------------------------------+-----------+------------------+--------------------------------------------------------------------------------+-------------------+------------------------------+-------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------

In [38]:
# number of rows in the data
print(len( df))

10


In [31]:
# function that extract the browser information
def extract_browser(device_dict):
    return device_dict.get('web_info', {}).get('browser', 'Unknown')

df['browser'] = df['device'].apply(extract_browser) # extract browser from 'device' column
browser_counts = df['browser'].value_counts()
print(browser_counts)

Chrome    10
Name: browser, dtype: int64


In [53]:
for index, device_dict in df['device'].items():
  if(index<4):# print just the first 4 
    print(f"--- Device {index + 1} ---")
    print(f"Category: {device_dict.
          get('category', '')}")
    print(f"Brand: {device_dict.get('mobile_brand_name', '')}")
    print(f"Model: {device_dict.get('mobile_model_name', '')}")
    print(f"Operating System: {device_dict.get('operating_system', '')}")
    print(f"Browser: {device_dict.get('web_info', {}).get('browser', '')}")
    print() 



--- Device 1 ---
Category: desktop
Brand: Google
Model: Chrome
Operating System: Web
Browser: Chrome

--- Device 2 ---
Category: desktop
Brand: Google
Model: Chrome
Operating System: Web
Browser: Chrome

--- Device 3 ---
Category: desktop
Brand: Google
Model: Chrome
Operating System: Web
Browser: Chrome

--- Device 4 ---
Category: desktop
Brand: Google
Model: Chrome
Operating System: Web
Browser: Chrome

